# Struggling with the computation complexity

In [1]:
import numpy as np
import pandas as pd
import ripser
import time
# import ripserplusplus as rpp_py
from persim.persistent_entropy import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neighbors import NearestNeighbors
import subprocess as sp
import gc

In [2]:
def get_gpu_memory():
    _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]

    ACCEPTABLE_AVAILABLE_MEMORY = 1024
    COMMAND = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = _output_to_list(sp.check_output(COMMAND.split()))[1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    #print("free GPU memory (mega): ",memory_free_values)
    print("free GPU memory (MiB):")
    print(memory_free_info[0])
    return memory_free_values

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

housing = fetch_california_housing()

X_all = pd.DataFrame(housing.data, columns = housing.feature_names)
y_all = housing['target']

X_all = X_all.iloc[:1000]
y_all = y_all[:1000]

In [4]:
scaler = StandardScaler()
scaler.fit(X_all)
X_scaled = pd.DataFrame(scaler.transform(X_all), index= X_all.index, columns= X_all.columns)
# X_final = X.iloc[:, :8].to_numpy()

X_scaled

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.475751,0.266201,1.500473,-0.266636,-0.900449,-0.219674,1.441017,-0.640477
1,2.462605,-1.251479,0.863503,-0.751121,1.186500,-0.830714,1.235934,-0.554449
2,1.885951,1.100925,2.613913,0.196463,-0.725783,0.118538,1.133393,-0.726504
3,0.994291,1.100925,0.504211,0.192853,-0.663546,-0.230108,1.133393,-0.812532
4,0.001771,1.100925,0.900830,0.267694,-0.656520,-0.732521,1.133393,-0.812532
...,...,...,...,...,...,...,...,...
995,0.563070,-2.010319,0.386932,-0.071678,4.624595,0.035443,-0.302190,3.488856
996,2.933043,-0.947943,0.499441,-0.927820,-0.968709,0.373497,0.005435,3.316800
997,0.493308,-1.175595,0.728664,-0.150441,1.259779,0.003959,-0.404732,3.058717
998,0.877910,-1.555015,0.639530,-0.812918,1.006816,0.878353,-0.507273,3.058717


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_all, test_size=0.5, random_state=43)

lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

RMSE: 0.5137944706615996
R-square: 0.6499318732684514


# Add Entropy featureX

### reset for the index drop

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

### Full ripser ++

In [39]:
entropy_feature = []

start = time.time()
for i in range(X_scaled.shape[0]):
    if (i + 1) % 30 == 0:
        print(f"Processing row {i + 1} out of {X_scaled.shape[0]}")
        
        get_gpu_memory()
        gc.collect()
        
    data_remove = X_scaled.drop(index=i)
    pca_result = pca.fit_transform(data_remove)
    d = rpp_py.run("--format point-cloud --dim 1 --threshold 1",pca_result)
    dgm = np.array([(b, d) for b, d in d[0]])
    p_entropy = persistent_entropy(dgm)
    
    entropy_feature.append(p_entropy)
    
end = time.time()
print("ripser++ total time: ", end-start)

NameError: name 'rpp_py' is not defined

In [10]:
X_scaled['topo_feature'] = [x.tolist()[0] if isinstance(x, np.ndarray) else x for x in entropy_feature]
X_scaled['topo_feature'] = scaler.fit_transform(X_scaled[['topo_feature']])
X_scaled.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,topo_feature
0,2.642119,0.828791,0.281990,-0.123927,-0.997996,-0.331205,0.202650,-0.604776,-0.207338
1,2.629035,-0.649917,0.111970,-0.179877,1.296598,-0.881856,0.189239,-0.598947,-0.056716
2,2.055091,1.642080,0.579190,-0.070446,-0.805952,-0.026418,0.182533,-0.610606,-0.213089
3,1.167623,1.642080,0.016067,-0.070863,-0.737523,-0.340607,0.182533,-0.616435,-0.127761
4,0.179770,1.642080,0.121933,-0.062220,-0.729797,-0.793367,0.182533,-0.616435,-0.084355


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_all, test_size=0.5, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = pd.Series(y_train).reset_index(drop=True)
y_test = pd.Series(y_test).reset_index(drop=True)

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

RMSE: 0.5065670208025156
R-square: 0.7211150838116289


## Parallel

In [6]:
from sklearn.metrics import pairwise_distances
import os
from joblib import Parallel, delayed

In [12]:
start = time.time()
data_list = []
for i in range(X_scaled.shape[0]):
    if (i + 1) % 500 == 0:
        print(f"Processing row {i + 1} out of {X_scaled.shape[0]}")
        
        gc.collect()
        
    data_remove = X_scaled.drop(index=i)
    # distances = pairwise_distances(data_remove, metric='euclidean')
    data_list.append(data_remove)
    
end = time.time()

print("parallel total time: ", end-start)

Processing row 500 out of 1000
Processing row 1000 out of 1000
parallel total time:  0.8318099975585938


In [13]:
def compute_distances(data, index):
    dist_matrix = pairwise_distances(data, metric='euclidean')
    return index, dist_matrix

dists = Parallel(n_jobs=-1)(delayed(compute_distances)(data, i) for i, data in enumerate(data_list))

In [14]:
def compute_persistence(data_remove):
    print(f"進程 {os.getpid()} 開始計算")
    
    pca_result = pca.fit_transform(data_remove)
    dgm = ripser.ripser(pca_result, maxdim = 0, distance_matrix=True)['dgms'][0]
    
    p_entropy = persistent_entropy(dgm)

    print(f"進程 {os.getpid()} 完成計算")
    
    return p_entropy

In [15]:
# data_list
distance_matrices = [pairwise_distances(data, metric='euclidean') for data in data_list]

In [14]:
if __name__ == "__main__":

    start_time = time.time()

    # 使用 joblib 進行平行運算
    results = Parallel(n_jobs=10)(delayed(compute_persistence)(data) for data in data_list)

    end_time = time.time()
    print("平行運算時間: ", end_time - start_time)

平行運算時間:  907.4925081729889


In [15]:
X_scaled['topo_feature'] = [x.tolist()[0] if isinstance(x, np.ndarray) else x for x in results]
X_scaled['topo_feature'] = scaler.fit_transform(X_scaled[['topo_feature']])
X_scaled.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,topo_feature
0,2.642119,0.828791,0.281990,-0.123927,-0.997996,-0.331205,0.202650,-0.604776,-0.071041
1,2.629035,-0.649917,0.111970,-0.179877,1.296598,-0.881856,0.189239,-0.598947,-0.019845
2,2.055091,1.642080,0.579190,-0.070446,-0.805952,-0.026418,0.182533,-0.610606,-0.060085
3,1.167623,1.642080,0.016067,-0.070863,-0.737523,-0.340607,0.182533,-0.616435,-0.037265
4,0.179770,1.642080,0.121933,-0.062220,-0.729797,-0.793367,0.182533,-0.616435,-0.032980


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_all, test_size=0.5, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = pd.Series(y_train).reset_index(drop=True)
y_test = pd.Series(y_test).reset_index(drop=True)

lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

RMSE: 0.50665474264537
R-square: 0.7210184868520162


## Testing

In [40]:
import numpy as np
from ripser import ripser
from scipy.spatial.distance import pdist, squareform

# 模擬資料
data = np.random.rand(20000, 2)  # 100個資料點，每個資料點有2個維度

# 預計算距離矩陣
distance_matrix = squareform(pdist(data))

def compute_partial_persistence_diagram(dist_matrix, remove_index, maxdim=0):
    # 移除某個資料點，重新計算拓撲結構
    dist_matrix_remove = np.delete(distance_matrix, remove_index, axis=0)
    dist_matrix_remove = np.delete(dist_matrix_remove, remove_index, axis=1)
    
    # 計算 persistent homology (僅計算 H0)
    result = rpp_py.run("--format distance --dim 1", dist_matrix_remove)
    return result[0]  # 回傳H0的Persistence Diagram

def leave_one_out_persistence(data, distance_matrix, maxdim=0):
    pdgms = []
    n_points = data.shape[0]  # 資料點數量
    for i in range(n_points):
        if (i + 1) % 50 == 0:
            print(f"Calculating persistence diagram by leaving out point {i+1}/{n_points}...")
        pdgm = compute_partial_persistence_diagram(distance_matrix, i, maxdim=maxdim)
        pdgms.append(pdgm)
    return pdgms

In [ ]:
start = time.time()

all_pdms = leave_one_out_persistence(data, distance_matrix, maxdim=0)
all_pdms

end = time.time()
print("ripser++ total time: ", end-start)